In [1]:
import math
import torch
import gpytorch
from matplotlib import pyplot as plt

%matplotlib inline
%load_ext autoreload
%autoreload 2

/home/q123/miniconda3/envs/explo/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Training data is 100 points in [0,1] inclusive regularly spaced
train_x = torch.linspace(0, 1, 100)
# True function is sin(2*pi*x) with Gaussian noise
train_y = torch.sin(train_x * (2 * math.pi)) + torch.randn(train_x.size()) * math.sqrt(0.04)

In [3]:
class MyKernel(gpytorch.kernels.RBFKernel):
    
   
    def forward(self,x1,x2,**params):
        
        print("x1.shape",x1.shape)
        rslt = super().forward(x1,x2,**params)
        print("pair kernel shape",rslt.shape)
        return rslt

In [4]:
# We will use the simplest form of GP model, exact inference
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(MyKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(train_x, train_y, likelihood)

In [5]:
# this is for running the notebook in our testing framework
import os
smoke_test = ('CI' in os.environ)
training_iter = 2 if smoke_test else 50


# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

for i in range(3):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(train_x)
    # Calc loss and backprop gradients
    loss = -mll(output, train_y)
    print("LOSS SHAPE",loss.size())
    loss.backward()
    print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
        i + 1, training_iter, loss.item(),
        model.covar_module.kernel.base_kernel.lengthscale.item(),
        model.likelihood.noise.item()
    ))
    optimizer.step()

x1.shape torch.Size([100, 1])
pair kernel shape torch.Size([100, 100])
LOSS SHAPE torch.Size([])
Iter 1/50 - Loss: 0.928   lengthscale: 0.693   noise: 0.693
x1.shape torch.Size([100, 1])
pair kernel shape torch.Size([100, 100])
LOSS SHAPE torch.Size([])
Iter 2/50 - Loss: 0.896   lengthscale: 0.644   noise: 0.644
x1.shape torch.Size([100, 1])
pair kernel shape torch.Size([100, 100])
LOSS SHAPE torch.Size([])
Iter 3/50 - Loss: 0.862   lengthscale: 0.598   noise: 0.598
